## Example to generate/edit/upload to `model.cc` file Google Drive.

Our implementation requires setting up a Google Cloud project as we use Google Drive as storage, authorization is done over `OAuth 2.0`.

Check the following page to learn how to set up a project: [Google's OAuth 2.0 protocols](https://developers.google.com/identity/protocols/oauth2).

The default `model.cc` need some tweaking. This is an example notebook, where example model is used that shows how to generate ready-to-use `model.cc` file.

This file contains our model and required parameters for scaling.

In [9]:
import os
import os.path
import logging
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from urllib.request import urlopen
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google_auth_oauthlib.flow import Flow
from googleapiclient.errors import HttpError

In [2]:
tf.get_logger().setLevel('ERROR')
logging.getLogger("tensorflow").setLevel(logging.ERROR)
# Fetch dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv"
data = pd.read_csv(urlopen(url))

# Preprocess data
# For simplicity, let's predict the temperature based on the last 5 days.
def create_dataset(X, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(X.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

time_steps = 5
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data[['Temp']].values)

# Create dataset for LSTM
X, y = create_dataset(pd.DataFrame(scaled_data), time_steps)

# Split dataset (let's use the first 80% of the data for training and the rest for testing)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Define LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.1, verbose=1)

2024-03-31 10:28:25.839939: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46646 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:65:00.0, compute capability: 8.6


Epoch 1/5


2024-03-31 10:28:28.146986: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f27899e6030 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-31 10:28:28.147016: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2024-03-31 10:28:28.151586: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-31 10:28:28.169126: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
I0000 00:00:1711880908.254985 1453555 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


82/82 [==============================] - 3s 12ms/step - loss: 0.0396 - val_loss: 0.0111
Epoch 2/5
82/82 [==============================] - 1s 9ms/step - loss: 0.0115 - val_loss: 0.0107
Epoch 3/5
82/82 [==============================] - 1s 10ms/step - loss: 0.0111 - val_loss: 0.0106
Epoch 4/5
82/82 [==============================] - 1s 10ms/step - loss: 0.0109 - val_loss: 0.0106
Epoch 5/5
82/82 [==============================] - 1s 10ms/step - loss: 0.0107 - val_loss: 0.0102


In [4]:
# Model directory to save the TensorFlow SavedModel
MODEL_DIR = "/tf/models/lstm_ex"
model.save(MODEL_DIR, save_format="tf")

# Convert the SavedModel to a TensorFlow Lite model
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)

# Enable Select TensorFlow operations
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS     # Enable TensorFlow ops.
]

# Disable lowering tensor list operations
converter._experimental_lower_tensor_list_ops = False

tflite_model = converter.convert()

# Path to save the TFLite model
tflite_model_path = "/tf/models/lstm_ex/model.tflite"

# Save the TFLite model to the file
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

# Get the size of the TFLite model file
tflite_file_size = os.path.getsize(tflite_model_path)

print(f"Model saved to: {tflite_model_path}")
print(f"Size of the TFLite model: {tflite_file_size} bytes")

Model saved to: /tf/models/lstm_ex/model.tflite
Size of the TFLite model: 52008 bytes


2024-03-31 10:32:15.876198: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-03-31 10:32:15.876229: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-03-31 10:32:15.876389: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tf/models/lstm_ex
2024-03-31 10:32:15.878403: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-03-31 10:32:15.878418: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tf/models/lstm_ex
2024-03-31 10:32:15.883733: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-03-31 10:32:15.921325: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tf/models/lstm_ex
2024-03-31 10:32:15.937818: I tensorflow/cc/saved_model/loader.cc:316] SavedModel load for tags { serve }; Status: success: OK. Took 6

You can ignore the above warnings. Now the below command generates our file but requires a tweaking.

In [12]:
!xxd -i /tf/models/lstm_ex/model.tflite > /tf/models/lstm_ex/model.cc

The required tweaks are:
- Including header file.
- Changing model parameter name.
- Adding scaling paramaters.

In actual scenario, the scaling parameters would be fetch from the main *training* dataset.

In [17]:
# Read the original file
with open('/tf/models/lstm_ex/model.cc', 'r') as file:
    lines = file.readlines()

# Flag to check if sensor values already exist
sensor_values_exist = any("ACCX_MEAN" in line for line in lines)

# Add the missing include at the beginning if not already present
if not any(line.strip() == '#include "model.h"' for line in lines):
    lines.insert(0, '#include "model.h"\n')

# Modify the variable declaration if needed
variable_declaration = 'alignas(16) const char g_model[] = {\n'
if not any(variable_declaration in line for line in lines):
    for i, line in enumerate(lines):
        if line.strip().startswith('unsigned char _tf_models_lstm_ex_model_tflite[]'):
            lines[i] = variable_declaration
            break

# Append sensor mean and standard deviation values at the end if they do not exist
if not sensor_values_exist:
    sensor_values = """
// Accelerometer X-axis
const float ACCX_MEAN = 9.235880f;    
const float ACCX_STD = 1.516386f;    

// Accelerometer Y-axis
const float ACCY_MEAN = -0.682526f;    
const float ACCY_STD = 3.083149f;      

// Accelerometer Z-axis
const float ACCZ_MEAN = 0.4874396f;    
const float ACCZ_STD = 0.234757f;      

// Gyroscope X-axis
const float GYROX_MEAN = 0.344743f;  
const float GYROX_STD = 28.02007f;    

// Gyroscope Y-axis
const float GYROY_MEAN = 0.666154f; 
const float GYROY_STD = 8.086139f;  

// Gyroscope Z-axis
const float GYROZ_MEAN = 0.011609f; 
const float GYROZ_STD = 26.38441f;      

// Magnetometer X-axis
const float MAGX_MEAN = -33.35714f;  
const float MAGX_STD = 5.843046f;    

// Magnetometer Y-axis
const float MAGY_MEAN = 19.60439f;  
const float MAGY_STD = 19.29037f;    

// Magnetometer Z-axis
const float MAGZ_MEAN = -2.507685f;  
const float MAGZ_STD = 31.47144f;
"""
    lines.append(sensor_values)

# Write the modified content back
with open('/tf/models/lstm_ex/model.cc', 'w') as file:
    file.writelines(lines)


This is how the authorization is done. 

Get your token if you are running for the first time, then repeated accesses will not require any.

In [6]:
SCOPES = ['https://www.googleapis.com/auth/drive.file']

creds = None
if os.path.exists('/tf/token.json'):
    creds = Credentials.from_authorized_user_file('/tf/token.json', SCOPES)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = Flow.from_client_secrets_file(
            '/tf/credentials.json', 
            scopes=SCOPES,
            redirect_uri='urn:ietf:wg:oauth:2.0:oob')
        
        auth_url, _ = flow.authorization_url(prompt='consent')

        print('Please go to this URL and authorize the app:')
        print(auth_url)
        auth_code = input('Enter the authorization code: ')
        flow.fetch_token(code=auth_code)
        
        creds = flow.credentials
        with open('/tf/token.json', 'w') as token_file:
            token_file.write(creds.to_json())

We then upload our file.

In [13]:
try:
    # Step 1: Set up the Drive service
    service = build('drive', 'v3', credentials=creds)

    # Step 2: Define file metadata and media content
    file_metadata = {'name': 'model.cc'}
    media = MediaFileUpload('/tf/models/lstm_ex/model.cc', mimetype='text/plain')
    
    # Step 3: Execute the upload
    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    
    # Print the uploaded file ID
    print(f'File ID: {file.get("id")}')
except HttpError as error:
    # Handle errors from the Drive API.
    print(f'An error occurred: {error}')

File ID: 105L9d3JU8n2AMqf8NiZbJGeVSa2_rVEs


Now our file ready in cloud to be downloaded to fog node where we do OTA.